# Welcome to Colab Enterprise <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">

Connect to a Runtime and begin!

<div class="markdown-google-sans">

## **Getting started**
</div>

##### Strictly Google Proprietary and Confidential

Author - Saurabhmangal@ + Rajeshthallam@

In [1]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    # app = IPython.Application.instance()
    # app.kernel.do_shutdown(True)

In [ ]:
# PROJECT_ID and all details need to be changed here


VERTEX_API_PROJECT = PROJECT_ID = "my-project-0004-346516" #'your-project' #@param {"type": "string"}

REGION = "europe-west4"
GCS_BUCKET = "my-project-0004-346516-vertex-pipelines-europe-west4"
SERVICE_ACCOUNT = "model-serve@cloud-nas-260507.iam.gserviceaccount.com"
SERVE_DOCKER_URI = f"{REGION}-docker.pkg.dev/cloud-nas-260507/pytorch/vllm-serve:latest"
SERVE_DOCKER_URI = VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve"

import os
from datetime import datetime


In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

In [ ]:
# from google.cloud import aiplatform
# aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

In [ ]:

def get_job_name_with_datetime(prefix: str):
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    model_name,
    model_id,
    service_account,
    machine_type="n1-highmem-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1,
):
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.9",
        "--disable-log-stats"
    ]
    model = aiplatform.Model.upload(
        display_name=model_name,
        # serving_container_image_uri="europe-west4-docker.pkg.dev/cloud-nas-260507/pytorch/vllm-serve:latest",
        serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve",
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        # service_account=service_account,
    )
    return model, endpoint


model, endpoint = deploy_model(
    model_name=get_job_name_with_datetime(prefix="mistral-7b"),
    model_id="mistralai/Mistral-7B-v0.1",
    service_account=SERVICE_ACCOUNT,
    machine_type="g2-standard-8",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/255766800726/locations/europe-west4/endpoints/5240118486352527360/operations/8257119294219878400
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/255766800726/locations/europe-west4/endpoints/5240118486352527360
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/255766800726/locations/europe-west4/endpoints/5240118486352527360')
INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/255766800726/locations/europe-west4/models/1845002501640683520/operations/763129514275373056
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/255766800726/locations/europe-west4/models/1845002501640683520@1
INFO:google.cloud.aiplatform.models:To use this Model 

In [ ]:
# predict_custom_trained_model_sample(
#     project="255766800726",
#     endpoint_id="8966847178001612800",
#     location="europe-west4",
#     instances={ "instance_key_1": "value", ...}
# )

In [ ]:
question = "Which are two men doing?"
instances = [
    {"prompt": question,
     "n":1,
     "max_tokens": 64},
]
response = endpoint.predict(instances=instances) #.predictions
print(response.predictions[0])


Prompt:
Which are two men doing?
Output:
A: One is playing football and another, Mallumble, is trying to forest the ball. But Mallumble does not succeed, Ortman catches the title of the ball to attempt a goal.

B: Two boys are playing touch football. In this version, they take balls into each other area
